In [14]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, cross_val_score
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sceli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# этот блок добавлен, потому что выполнял на ПК Windows, чтобы не ошибиться и не переписывать для работы на серере
path_data_train = os.path.normpath('datasets/tweets_lemm_train.csv')
path_data_test = os.path.normpath('datasets/tweets_lemm_test.csv')

In [3]:
data_train = pd.read_csv(path_data_train)
data_test = pd.read_csv(path_data_test)

In [4]:
data_train.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5000 non-null   object
 1   positive   5000 non-null   int64 
 2   lemm_text  5000 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [6]:
data_test.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [7]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3000 non-null   object
 1   lemm_text  3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


In [8]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf = count_tf_idf.fit_transform(list(data_train['lemm_text']))

In [9]:
tf_idf.shape

(5000, 9737)

In [10]:
count_tf_idf_2 = TfidfVectorizer(stop_words=stop_words, ngram_range=(2,2))
tf_idf_2 = count_tf_idf_2.fit_transform(list(data_train['lemm_text']))

In [11]:
tf_idf_2.shape

(5000, 25443)

In [18]:
molel_logistic_1 = LogisticRegression()
molel_logistic_2 = LogisticRegression()

In [19]:
cross_val_score(molel_logistic_1, tf_idf.toarray(), data_train['positive'])

array([0.633, 0.666, 0.646, 0.629, 0.607])

In [20]:
cross_val_score(molel_logistic_2, tf_idf_2.toarray(), data_train['positive'])

array([0.539, 0.545, 0.538, 0.548, 0.539])

In [21]:
molel_logistic_1.fit(tf_idf.toarray(), data_train['positive'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
features_test = count_tf_idf.transform(list(data_test['lemm_text']))

In [24]:
outfile = molel_logistic_1.predict(features_test.toarray())

In [29]:
pd.DataFrame(outfile, columns=['positive']).to_csv('predict', index = False)